In [1]:
import requests
import pandas as pd
from io import StringIO
import json

def fetch_destatis_data(username, password,
                       table_id,
                       area='all',
                       format='csv',
                       startyear=None,
                       endyear=None,
                       timeslices=None,
                       regionalvariable=None,
                       regionalkey=None,
                       classifyingvariable1=None,
                       classifyingkey1=None,
                       classifyingvariable2=None,
                       classifyingkey2=None,
                       classifyingvariable3=None,
                       classifyingkey3=None,
                       language='de'):
    """
    Fetches data from GENESIS database and returns pandas DataFrame

    Parameters:
    -----------
    username : str
        GENESIS API username
    password : str
        GENESIS API password
    table_id : str
        ID of the table to fetch (default: '51000-0014')
    area : str
        Area to fetch data from (default: 'all')
    format : str
        Data format (default: 'csv')
    startyear : str
        Start year for data (format: YYYY)
    endyear : str
        End year for data (format: YYYY)
    timeslices : str
        Number of time slices
    classifyingvariable1 : str
        For table 51000-0014, use 'WAM8' for Warenverzeichnis
    classifyingkey1 : str
        Warenverzeichnis code (e.g., 'WA09021000')
    language : str
        Language for results (default: 'de')

    Returns:
    --------
    pd.DataFrame
        DataFrame containing the requested data
    """

    # API endpoint
    base_url = "https://www-genesis.destatis.de/genesisWS/rest/2020/data/tablefile"

    # Build parameters dictionary
    params = {
        'username': username,
        'password': password,
        'name': table_id,
        'area': area,
        'format': format,
        'language': language,
        'startyear': startyear if startyear else '',
        'endyear': endyear if endyear else '',
        'timeslices': timeslices if timeslices else '',
        'regionalvariable': regionalvariable if regionalvariable else '',
        'regionalkey': regionalkey if regionalkey else '',
        'classifyingvariable1': classifyingvariable1 if classifyingvariable1 else '',
        'classifyingkey1': classifyingkey1 if classifyingkey1 else '',
        'classifyingvariable2': classifyingvariable2 if classifyingvariable2 else '',
        'classifyingkey2': classifyingkey2 if classifyingkey2 else '',
        'classifyingvariable3': classifyingvariable3 if classifyingvariable3 else '',
        'classifyingkey3': classifyingkey3 if classifyingkey3 else '',
        'compress': 'false',
        'transpose': 'false'
    }

    try:
        # Make API request
        response = requests.get(base_url, params=params)

        # Check if response is JSON (error message)
        try:
            error_data = response.json()
            if error_data.get('Status', {}).get('Type') == 'Fehler':
                print(f"API Error: {error_data['Status']['Content']}")
                return None
        except json.JSONDecodeError:
            pass  # Response is not JSON, continue processing as CSV

        # Extract CSV data from response
        csv_data = response.text

        # Check if response contains actual data
        if 'GENESIS-Tabelle' not in csv_data:
            print("No data found in response")
            return None

        # Convert to DataFrame
        # Skip first few rows which contain metadata and find actual data start
        lines = csv_data.split('\n')
        data_start = 0
        for i, line in enumerate(lines):
            if any(line.startswith(x) for x in [';', 'Zeit', 'Jahr', 'Monat']):
                data_start = i
                break

        # Read CSV data starting from actual data rows
        df = pd.read_csv(StringIO('\n'.join(lines[data_start:])),
                        sep=';',
                        decimal=',',
                        thousands='.',
                        na_values=['-', '.', '...'])

        # Clean up column names
        df.columns = df.columns.str.strip()

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None


In [5]:

username = ""
password = ""


# With filters
df = fetch_destatis_data(
    username,
    password,
    table_id='43511-0001',
    startyear='2021',
    endyear='2021',
    classifyingvariable2="MONAT",
    classifyingkey2="MONAT01",
)

In [6]:
df

,Unnamed: 0,Unnamed: 1,Einfuhr von Steinkohle in Tonnen,Einfuhr von Steinkohle in Gigajoule,Durchschnittlicher Heizwert von Steinkohle (Hi),Einfuhr von Steinkohle in Euro,Einfuhr von Steinkohle in Euro/t,Einfuhr von Steinkohle in Euro/tSKE
0,NaN,NaN,t,GJ,MJ/t,EUR,EUR/t,EUR/tSKE
1,2021,Januar,2910831,81047424,27843,224393756,"77,09","81,14"
2,__________,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Â© Statistisches Bundesamt (Destatis), 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Stand: 30.10.2024 / 15:39:34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
